# Install and Import Qiskit
This section will help you install Qiskit (if needed) and import its main modules.

In [99]:
# If running in an environment where Qiskit is not installed, uncomment the next line:

from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
#from qiskit.visualization import plot_histogram

from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import PassManager
from qiskit.providers.fake_provider import GenericBackendV2
import time
from tabulate import tabulate
from qiskit.circuit.random import random_circuit
from qiskit.transpiler.passes import (
    RemoveResetInZeroState,
    RemoveDiagonalGatesBeforeMeasure,
    Optimize1qGates,
    CommutationAnalysis,
    CommutativeCancellation,
    Collect2qBlocks,
    ConsolidateBlocks,
    UnitarySynthesis,
)

# Check Qiskit Version
Print the installed Qiskit version to confirm the environment setup.

In [100]:
import qiskit
print("Qiskit version:", qiskit.__version__)

Qiskit version: 2.1.0


# Create the service provider
Create a service provider instance to manage the quantum backend and other configurations.

In [101]:
# Initialize the Qiskit Runtime Service
service = QiskitRuntimeService(
    channel="ibm_quantum",
    instance="ibm-q/open/main",  # ="<IBM Cloud CRN or instance name>", # Optionally specify the instance to use.
    token="6eeda133df82dbaa57e0b308e80950f4452e32b1b9d0e942d90986e0547c24af26a2f5b5da5ed30b46d4263a938e5d3ac46fabe1ec59bc527e0ba7a0ea2184d6",
)
# List all available backends
backends = service.backends()
print("Available backends:")
for backend in backends:
    print(f"- {backend.name} ({backend.status})")

/tmp/ipykernel_106354/3458285426.py:2: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud", "ibm_quantum_platform", and "local" will be the only valid channels. Open Plan users should migrate now.  All other users should review the migration guide (https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp)to learn when to migrate.
  service = QiskitRuntimeService(


Available backends:
- ibm_brisbane (<bound method IBMBackend.status of <IBMBackend('ibm_brisbane')>>)
- ibm_sherbrooke (<bound method IBMBackend.status of <IBMBackend('ibm_sherbrooke')>>)


```markdown
# Preprocessing Quantum Circuits with Transpiler Passes

This cell defines a function `preprocess_circuit` that applies a sequence of Qiskit transpiler passes to optimize and simplify a given quantum circuit. The function uses passes such as removing resets in the zero state, eliminating diagonal gates before measurement, optimizing single-qubit gates, analyzing and canceling commutative gates, collecting two-qubit blocks, and consolidating blocks. Optionally, it can also synthesize unitary gates into a specified set of basis gates for further optimization.
```

In [102]:
def preprocess_circuit(qc: QuantumCircuit, basis_gates=None, basic_passes = True) -> QuantumCircuit:
    pm = PassManager()
    pm.append(RemoveResetInZeroState())
    pm.append(RemoveDiagonalGatesBeforeMeasure())
    pm.append(Optimize1qGates())
    pm.append(CommutationAnalysis())
    pm.append(CommutativeCancellation())
    pm.append(Collect2qBlocks())
    pm.append(ConsolidateBlocks())
    # The following passes are commented out as they may not be necessary for all circuits.
    # Uncomment them if you want to include them in your preprocessing pipeline.
    # RemoveBarriers removes barriers from the circuit, which can be useful for optimization.
    # CXCancellation removes controlled-X gates that are not needed, which can simplify the circuit.
    # RemoveUnusedRegisters removes any unused quantum or classical registers from the circuit.
    if not basic_passes:
        from qiskit.transpiler.passes import RemoveBarriers, Optimize1qGatesDecomposition, Collect1qRuns, CollectMultiQBlocks, CollectAndCollapse, CollectLinearFunctions, CollectCliffords, InverseCancellation, CommutativeInverseCancellation, Optimize1qGatesSimpleCommutation, RemoveFinalReset, HoareOptimizer, TemplateOptimization, ResetAfterMeasureSimplification, OptimizeCliffords, ElidePermutations, OptimizeAnnotated, Split2QUnitaries, RemoveIdentityEquivalent, ContractIdleWiresInControlFlow, OptimizeCliffordT
        pm.append(RemoveBarriers())
        pm.append(Optimize1qGatesDecomposition())
        pm.append(Collect1qRuns())
        pm.append(CollectMultiQBlocks())
        #pm.append(CollectAndCollapse())
        pm.append(CollectLinearFunctions())
        #pm.append(CollectCliffords())
        #pm.append(InverseCancellation())
        pm.append(CommutativeInverseCancellation())
        pm.append(Optimize1qGatesSimpleCommutation())
        pm.append(RemoveFinalReset())
        pm.append(HoareOptimizer())
        pm.append(TemplateOptimization())
        pm.append(ResetAfterMeasureSimplification())
        pm.append(OptimizeCliffords())
        pm.append(ElidePermutations())
        pm.append(OptimizeAnnotated())
        pm.append(Split2QUnitaries())
        pm.append(RemoveIdentityEquivalent())
        pm.append(ContractIdleWiresInControlFlow())
        pm.append(OptimizeCliffordT())

    # Optionally add UnitarySynthesis if basis_gates are provided
    # This pass synthesizes unitary gates into the specified basis gates.
    # It can be useful for optimizing the circuit further, especially if you have a specific gate
    # set you want to target for execution.
    # If you don't specify basis_gates, it will use the default basis gates of the backend.
    if basis_gates:
        pm.append(UnitarySynthesis(basis_gates=basis_gates))
    return pm.run(qc)

In [103]:

def build_random_test_circuit(n_qubits: int) -> QuantumCircuit:
    # depth=10 is arbitrary; adjust as needed
    qc = random_circuit(n_qubits, depth=10, max_operands=3, measure=True)#, seed=42)
    return qc

In [115]:
num_qubits = 50

# Use GenericBackendV2 as an example IBM backend
# backend = GenericBackendV2(num_qubits=num_qubits)

backend = service.backend("ibm_brisbane")
seed_sim = 42
backend = AerSimulator.from_backend(backend, seed_simulator=seed_sim)
print(f"Backend: {backend.name}")
basis = backend._basis_gates()

# 1) Create original
orig = build_random_test_circuit(num_qubits)

# 2) Direct full transpile (opt level 0)
start_time = time.time()
direct_0 = transpile(orig, backend, optimization_level=0)
print(f"Direct transpile opt level 0 took {time.time() - start_time:.3f} seconds")

# 2) Direct full transpile (opt level 1)
start_time = time.time()
direct_1 = transpile(orig, backend, optimization_level=1)
print(f"Direct transpile opt level 1 took {time.time() - start_time:.3f} seconds")

# 2) Direct full transpile (opt level 2)
start_time = time.time()
direct_2 = transpile(orig, backend, optimization_level=2)
print(f"Direct transpile opt level 2 took {time.time() - start_time:.3f} seconds")

# 3) Preprocess + transpile (opt level 0)
start_time = time.time()
pre = preprocess_circuit(orig, basis_gates=basis, basic_passes=False)
after_pre_0 = transpile(pre, backend, optimization_level=0)
print(f"Preprocess + transpile 0 took {time.time() - start_time:.3f} seconds")

# 3) Preprocess + transpile (opt level 1)
start_time = time.time()
pre = preprocess_circuit(orig, basis_gates=basis, basic_passes=False)
after_pre_1 = transpile(pre, backend, optimization_level=1)
print(f"Preprocess + transpile 1 took {time.time() - start_time:.3f} seconds")

# 3) Preprocess + transpile (opt level 2)
start_time = time.time()
pre = preprocess_circuit(orig, basis_gates=basis, basic_passes=False)
after_pre_2 = transpile(pre, backend, optimization_level=2)
print(f"Preprocess + transpile 2 took {time.time() - start_time:.3f} seconds")

# 4) Gather metrics
def metrics(circ):
    return {
        "depth": circ.depth(),
        "total_gates": len(circ.data),
        "total_ops": sum(circ.count_ops().values()),
        "cx_count": circ.count_ops().get("cx", 0),
        "h_count": circ.count_ops().get("h", 0),
        "x_count": circ.count_ops().get("x", 0),
        "measure_count": circ.count_ops().get("measure", 0),
        # "reset_count": circ.count_ops().get("reset", 0),
        # "barrier_count": circ.count_ops().get("barrier", 0),
        "swap_count": circ.count_ops().get("swap", 0),
        # "unitary_count": circ.count_ops().get("unitary", 0),
        # "reset_in_zero_count": circ.count_ops().get("reset_in_zero_state", 0),
        # "diagonal_gates_before_measure_count": circ.count_ops().get("diagonal_gates_before_measure", 0),
        # "optimize_1q_gates_count": circ.count_ops().get("optimize_1q_gates", 0),
        # "commutation_analysis_count": circ.count_ops().get("commutation_analysis", 0,
        # "commutative_cancellation_count": circ.count_ops().get("commutative_cancellation", 0),
        # "collect_2q_blocks_count": circ.count_ops().get("collect_2q_blocks", 0),
        # "consolidate_blocks_count": circ.count_ops().get("consolidate_blocks", 0),
        # "unitary_synthesis_count": circ.count_ops().get("unitary_synthesis", 0),
        "num_qubits": circ.num_qubits,
        "num_clbits": circ.num_clbits,
        "num_qubits_used": len(circ.qubits),
        "num_clbits_used": len(circ.clbits),
        "num_cregs_used": len(circ.cregs),
        "num_qregs_used": len(circ.qregs),
        # "num_parameters": len(circ.parameters),
        # "num_ancillas": len(circ.ancillas),
        # "num_ancillass": circ.num_ancillas,
    }

m_orig = metrics(orig)
m_direct_0 = metrics(direct_0)
m_direct_1 = metrics(direct_1)
m_direct_2 = metrics(direct_2)
m_after_pre_0 = metrics(after_pre_0)
m_after_pre_1 = metrics(after_pre_1)
m_after_pre_2 = metrics(after_pre_2)

# 5) Print comparison
print("\n--- Metrics Comparison ---")
headers = [
    "Metric",
    "Original",
    "Direct_0",
    "Direct_1",
    "Direct_2",
    "Pre+0",
    "Pre+1",
    "Pre+2",
]
rows = []
all_keys = m_orig.keys() #sorted(
    #set(
    #    m_orig.keys()
    #    | m_direct_0.keys()
    #    | m_direct_1.keys()
    #    | m_direct_2.keys()
    #    | m_after_pre_0.keys()
    #    | m_after_pre_1.keys()
    #    | m_after_pre_2.keys()
    #)
#)
for key in all_keys:
    rows.append(
        [
            key,
            m_orig.get(key, ""),
            m_direct_0.get(key, ""),
            m_direct_1.get(key, ""),
            m_direct_2.get(key, ""),
            m_after_pre_0.get(key, ""),
            m_after_pre_1.get(key, ""),
            m_after_pre_2.get(key, ""),
        ]
    )

Backend: aer_simulator_from(ibm_brisbane)
Direct transpile opt level 0 took 0.185 seconds
Direct transpile opt level 1 took 0.342 seconds
Direct transpile opt level 2 took 0.903 seconds
Preprocess + transpile 0 took 2.018 seconds
Preprocess + transpile 1 took 2.067 seconds
Preprocess + transpile 2 took 2.611 seconds

--- Metrics Comparison ---


In [116]:
print(tabulate(rows, headers=headers, tablefmt="grid"))

+-----------------+------------+------------+------------+------------+---------+---------+---------+
| Metric          |   Original |   Direct_0 |   Direct_1 |   Direct_2 |   Pre+0 |   Pre+1 |   Pre+2 |
+=================+============+============+============+============+=========+=========+=========+
| depth           |         11 |       3885 |       1974 |       1762 |    3299 |    1290 |    1291 |
+-----------------+------------+------------+------------+------------+---------+---------+---------+
| total_gates     |        376 |      38641 |      12949 |      12732 |   27952 |    8282 |    7829 |
+-----------------+------------+------------+------------+------------+---------+---------+---------+
| total_ops       |        376 |      38641 |      12949 |      12732 |   27952 |    8282 |    7829 |
+-----------------+------------+------------+------------+------------+---------+---------+---------+
| cx_count        |          0 |          0 |          0 |          0 |       0 | 

In [106]:

# print("\n-- Directly Transpiled Circuit --")
# direct_0.draw()
# direct_1.draw()
# direct_2.draw()
# print("\n-- Preprocessed + Transpiled Circuit --")
# after_pre_0.draw()
# after_pre_1.draw()
# after_pre_2.draw()

